In [6]:
# Import the research library 
import logging
from benchmarklib import BenchmarkDatabase, setup_logging
from benchmarklib.algorithms import GroverConfig, GroverRunner, calculate_grover_iterations

setup_logging(logging.DEBUG)

from benchmarklib import CliqueProblem, CliqueTrial 

db = BenchmarkDatabase("BatchClique.db", CliqueProblem, CliqueTrial)

benchmarklib.core.database - INFO - Database initialized: BatchClique.db (CLIQUE)


In [7]:
# Load Qiskit
from qiskit_ibm_runtime import QiskitRuntimeService, Batch
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv()
API_TOKEN = os.getenv("API_TOKEN")
API_INSTANCE = os.getenv("API_INSTANCE", None)
service = QiskitRuntimeService(channel="ibm_cloud", token=API_TOKEN, instance=API_INSTANCE)
backend = service.backend(name="ibm_rensselaer")

In [8]:
# Configure Grover Benchmark
config = GroverConfig(shots=10**4, optimization_level=3, dynamical_decoupling=True)
runner = GroverRunner(db_manager=db, service=service, backend=backend, config=config)

benchmarklib.algorithms.grover - INFO - GroverRunner initialized for CLIQUE problems


In [9]:
nodes_range = range(3, 7)
edge_prob_range = range(51, 100, 5)
#edge_prob_range = range(70, 71)

In [10]:
# benchmark driver

from benchmarklib.compilers import XAGCompiler, TruthTableCompiler, ClassiqCompiler

for compiler in [ClassiqCompiler()]:
    runner.start_batch(compiler)
    for nodes in nodes_range:
        for prob in edge_prob_range:
            size_filter = {'num_vertices': nodes, 'edge_probability': prob}
            for problem in db.find_problem_instances(
                size_filters=size_filter, 
                limit=10, 
                choose_untested=True,
                random_sample=True
            ):
                print(problem)
                target_clique_size = max(nodes//2, 2)
                cliques_of_target_size = problem.clique_counts[target_clique_size]
                if cliques_of_target_size == 0:
                    # clique of size target_clique_size DNE for this graph
                    continue
    
                optimal_grover_iters = calculate_grover_iterations(cliques_of_target_size, 2**nodes)
                print(f"target_clique_size: {target_clique_size} \t cliques_of_target_size: {cliques_of_target_size} \t optimal_iters: {optimal_grover_iters}")
                for grover_iter in range(1, optimal_grover_iters+1):

                    # we have a trial for this problem and for each possible grover 
                    # iteration up to the optimal and for each compiler type
    
                    # saved to DB by GroverRunner
                    runner.run_grover_benchmark(
                        problem_instance=problem,
                        compiler=compiler,
                        grover_iterations=grover_iter,
                        # kwargs for oracle compilation
                        clique_size=target_clique_size
                    )
                    
        # Submit all circuits for this num_vars as one job
        job_id = runner.submit_job()
        print(f"Submitted job {job_id} for {compiler.name}, {nodes} nodes and {prob}% edge probability")
    
    runner.finish_batch()
            

benchmarklib.algorithms.grover - INFO - Starting batch for CLASSIQ
benchmarklib.algorithms.grover - DEBUG - Collecting circuit for Problem Instance ID: 1132


CLIQUE(num_vertices=3, num_edges=0, edge_probability=51)
CLIQUE(num_vertices=3, num_edges=3, edge_probability=51)
Debug: M=4, N=8, ratio=0.500, angle=0.785, result=1.000, floor=0, round=1
target_clique_size: 2 	 cliques_of_target_size: 4 	 optimal_iters: 1
COMPILING
1


benchmarklib.algorithms.grover - DEBUG - Simulation completed: 8 unique outcomes
benchmarklib.core.database - DEBUG - Saved new trial ID: 1475
benchmarklib.algorithms.grover - DEBUG - Saved trial 1475 (pending batch submission)
benchmarklib.algorithms.grover - DEBUG - Collected circuit 1 for batch (Problem 1132, 1 iterations)
benchmarklib.algorithms.grover - DEBUG - Collecting circuit for Problem Instance ID: 1172


CLIQUE(num_vertices=3, num_edges=0, edge_probability=61)
CLIQUE(num_vertices=3, num_edges=3, edge_probability=61)
Debug: M=4, N=8, ratio=0.500, angle=0.785, result=1.000, floor=0, round=1
target_clique_size: 2 	 cliques_of_target_size: 4 	 optimal_iters: 1
COMPILING
1


benchmarklib.algorithms.grover - DEBUG - Simulation completed: 8 unique outcomes
benchmarklib.core.database - DEBUG - Saved new trial ID: 1476
benchmarklib.algorithms.grover - DEBUG - Saved trial 1476 (pending batch submission)
benchmarklib.algorithms.grover - DEBUG - Collected circuit 2 for batch (Problem 1172, 1 iterations)
benchmarklib.algorithms.grover - DEBUG - Collecting circuit for Problem Instance ID: 1220


...

Submitted job d2j0uocg59ks73c391u0 for CLASSIQ, 5 nodes and 96% edge probability
CLIQUE(num_vertices=6, num_edges=6, edge_probability=51)
CLIQUE(num_vertices=6, num_edges=15, edge_probability=91)
Debug: M=42, N=64, ratio=0.656, angle=0.944, result=0.832, floor=0, round=1
target_clique_size: 3 	 cliques_of_target_size: 42 	 optimal_iters: 1
COMPILING
1


benchmarklib.algorithms.grover - ERROR - Simulation failed: 'Number of qubits (69) in circuit-10685 is greater than maximum (30) in the coupling_map'
benchmarklib.core.database - DEBUG - Saved new trial ID: 1515
benchmarklib.algorithms.grover - DEBUG - Saved trial 1515 (pending batch submission)
benchmarklib.algorithms.grover - DEBUG - Collected circuit 1 for batch (Problem 2662, 1 iterations)
benchmarklib.algorithms.grover - DEBUG - Collecting circuit for Problem Instance ID: 196


CLIQUE(num_vertices=6, num_edges=15, edge_probability=91)
Debug: M=42, N=64, ratio=0.656, angle=0.944, result=0.832, floor=0, round=1
target_clique_size: 3 	 cliques_of_target_size: 42 	 optimal_iters: 1
COMPILING
1


benchmarklib.algorithms.grover - ERROR - Simulation failed: 'Number of qubits (69) in circuit-10915 is greater than maximum (30) in the coupling_map'
benchmarklib.core.database - DEBUG - Saved new trial ID: 1516
benchmarklib.algorithms.grover - DEBUG - Saved trial 1516 (pending batch submission)
benchmarklib.algorithms.grover - DEBUG - Collected circuit 2 for batch (Problem 196, 1 iterations)
benchmarklib.algorithms.grover - INFO - Submitting job with 2 circuits
benchmarklib.algorithms.grover - DEBUG - Transpiling 2 circuits...
benchmarklib.core.database - DEBUG - Updated trial ID: 1515
benchmarklib.core.database - DEBUG - Updated trial ID: 1516
benchmarklib.algorithms.grover - INFO - Submitted job d2j0v0kg59ks73c39260 with 2 circuits (Job 4 in batch)
benchmarklib.algorithms.grover - INFO - Finished batch for CLASSIQ (4 jobs submitted)


Submitted job d2j0v0kg59ks73c39260 for CLASSIQ, 6 nodes and 96% edge probability
